In [32]:
# import modules
import json
import pandas as pd
import re
import regex
from datetime import datetime
pd.set_option("display.max_rows", 999)
pd.set_option("display.max_columns", 999)

import spacy
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize.toktok import ToktokTokenizer
import re
from bs4 import BeautifulSoup
import unicodedata

%reload_ext autoreload
%autoreload 2
from contractions import *

# nlp = spacy.load('en_core', parse=True, tag=True, entity=True)
# #nlp_vec = spacy.load('en_vecs', parse = True, tag=True, #entity=True)
# tokenizer = ToktokTokenizer()
# stopword_list = nltk.corpus.stopwords.words('english')
# stopword_list.remove('no')
# stopword_list.remove('not')

In [2]:
# day_count = (end_date - start_date).days + 1
# for single_date in (start_date + timedelta(n) for n in range(day_count)):
#     print strftime("%Y-%m-&d", single_date.timetuple()) 
    
#     filenames = '2016-07-' + str()

In [3]:

tw = pd.read_json('data/2020-03-20.json')
tw

,id,screen_name,user_id,time,link,text,source
0,1241177108442947584,brianschatz,47747074,2020-03-20T21:37:37-04:00,https://www.twitter.com/brianschatz/statuses/1...,@JNTHN_LCKWD @jbarro @CoryBooker Understood. W...,Twitter for iPhone
1,1241048506468622336,PattyMurray,293131808,2020-03-20T13:06:36-04:00,https://www.twitter.com/PattyMurray/statuses/1...,In accordance with current public health recom...,Twitter Web App
2,1241139330392961024,amyklobuchar,33537967,2020-03-20T19:07:30-04:00,https://www.twitter.com/amyklobuchar/statuses/...,The American people should know if they're sic...,Twitter Web App
3,1241044145994715136,dccc,14676022,2020-03-20T12:49:16-04:00,https://www.twitter.com/Court_Rice/statuses/12...,RT @Court_Rice LEADERSHIP: @CartwrightPA launc...,Twitter Web App
4,1241157748408176640,garretgraves,587100576,2020-03-20T20:20:41-04:00,https://www.twitter.com/garretgraves/statuses/...,https://www.wbrz.com/news/wbrz-business-direct...,Twitter for iPhone
...,...,...,...,...,...,...,...
4684,1241065460478676992,JGO_2020,3857977018,2020-03-20T14:13:58-04:00,https://www.twitter.com/Jenniffer2012/statuses...,RT @Jenniffer2012 Esto hará disponible más rec...,Twitter for iPhone
4685,1241065450966077440,JGO_2020,3857977018,2020-03-20T14:13:56-04:00,https://www.twitter.com/Jenniffer2012/statuses...,RT @Jenniffer2012 Envié una carta a la Adminis...,Twitter for iPhone
4686,1241057746050580480,JGO_2020,3857977018,2020-03-20T13:43:19-04:00,https://www.twitter.com/RepJenniffer/statuses/...,RT @RepJenniffer Sent a letter to @SBAgov Admi...,Twitter for iPhone
4687,1241053140973166592,JGO_2020,3857977018,2020-03-20T13:25:01-04:00,https://www.twitter.com/Jenniffer2012/statuses...,RT @Jenniffer2012 Excelente noticia del tesoro...,Twitter for iPhone


In [4]:
df = tw.drop(columns=["link", "source", "user_id"], axis=1)

In [5]:
df.head()

,id,screen_name,time,text
0,1241177108442947584,brianschatz,2020-03-20T21:37:37-04:00,@JNTHN_LCKWD @jbarro @CoryBooker Understood. W...
1,1241048506468622336,PattyMurray,2020-03-20T13:06:36-04:00,In accordance with current public health recom...
2,1241139330392961024,amyklobuchar,2020-03-20T19:07:30-04:00,The American people should know if they're sic...
3,1241044145994715136,dccc,2020-03-20T12:49:16-04:00,RT @Court_Rice LEADERSHIP: @CartwrightPA launc...
4,1241157748408176640,garretgraves,2020-03-20T20:20:41-04:00,https://www.wbrz.com/news/wbrz-business-direct...


In [6]:
format = "%Y-%m-%dT%H:%M:%S%z"
df.time = df.time.apply(lambda x: datetime.strptime(x, format))
df

,id,screen_name,time,text
0,1241177108442947584,brianschatz,2020-03-20 21:37:37-04:00,@JNTHN_LCKWD @jbarro @CoryBooker Understood. W...
1,1241048506468622336,PattyMurray,2020-03-20 13:06:36-04:00,In accordance with current public health recom...
2,1241139330392961024,amyklobuchar,2020-03-20 19:07:30-04:00,The American people should know if they're sic...
3,1241044145994715136,dccc,2020-03-20 12:49:16-04:00,RT @Court_Rice LEADERSHIP: @CartwrightPA launc...
4,1241157748408176640,garretgraves,2020-03-20 20:20:41-04:00,https://www.wbrz.com/news/wbrz-business-direct...
...,...,...,...,...
4684,1241065460478676992,JGO_2020,2020-03-20 14:13:58-04:00,RT @Jenniffer2012 Esto hará disponible más rec...
4685,1241065450966077440,JGO_2020,2020-03-20 14:13:56-04:00,RT @Jenniffer2012 Envié una carta a la Adminis...
4686,1241057746050580480,JGO_2020,2020-03-20 13:43:19-04:00,RT @RepJenniffer Sent a letter to @SBAgov Admi...
4687,1241053140973166592,JGO_2020,2020-03-20 13:25:01-04:00,RT @Jenniffer2012 Excelente noticia del tesoro...


In [7]:
df2 = df.text
df2[0]

'@JNTHN_LCKWD @jbarro @CoryBooker Understood. Working this hard. Don’t know what the result will be.'

In [8]:
df2[6]

'RT @RepFletcher When it comes to combating #COVID19, Texans should not have to worry about their insurance status. Today, I joined  @RepLloydDoggett and our Texas colleagues to call on Governor Abbot to expand coverage for #COVID19 care for Texans. http://pbs.twimg.com/media/ETmLzjqXYAAfhDV.png http://pbs.twimg.com/media/ETmLz4BXkAA0faZ.png'

In [9]:
df2[2]

'The American people should know if they\'re sick or not. \n\nThat’s the only way we can know if people developed immunities—then we will know which workers can be back on the front lines, who can give plasma to other people, &amp; much more. \n\nWe need to get more testing for the people. https://twitter.com/esmemurphy/status/1240752676691947527 QT @esmemurphy .@amyklobuchar says she finds lack of access to testing "appalling" says her office is getting large numbers of constituent calls needing help with Covid-19 concerns @wcco 5 and 6 pm'

In [10]:
df2[7]

'@bubbadog1 And hockey!'

## Count numbere of words

In [11]:
def num_of_words(df, col):
    df['word_count'] = df[col].apply(lambda x: len(str(x).split(" ")))
    print(df[[col, 'word_count']])

In [12]:
num_of_words(df, 'text')

                                                   text  word_count
0     @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...          14
1     In accordance with current public health recom...          40
2     The American people should know if they're sic...          85
3     RT @Court_Rice LEADERSHIP: @CartwrightPA launc...          29
4     https://www.wbrz.com/news/wbrz-business-direct...           1
...                                                 ...         ...
4684  RT @Jenniffer2012 Esto hará disponible más rec...          28
4685  RT @Jenniffer2012 Envié una carta a la Adminis...          73
4686  RT @RepJenniffer Sent a letter to @SBAgov Admi...          43
4687  RT @Jenniffer2012 Excelente noticia del tesoro...          58
4688  RT @Jenniffer2012 Esta mañana tuve un conferen...          42

[4689 rows x 2 columns]


## Count number of characters

In [13]:
def num_of_chars(df, col):
    df['char_count'] = df[col].str.len()
    print(df[[col, 'char_count']])

In [14]:
num_of_chars(df, 'text')

                                                   text  char_count
0     @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...          99
1     In accordance with current public health recom...         270
2     The American people should know if they're sic...         540
3     RT @Court_Rice LEADERSHIP: @CartwrightPA launc...         298
4     https://www.wbrz.com/news/wbrz-business-direct...          76
...                                                 ...         ...
4684  RT @Jenniffer2012 Esto hará disponible más rec...         185
4685  RT @Jenniffer2012 Envié una carta a la Adminis...         604
4686  RT @RepJenniffer Sent a letter to @SBAgov Admi...         343
4687  RT @Jenniffer2012 Excelente noticia del tesoro...         414
4688  RT @Jenniffer2012 Esta mañana tuve un conferen...         289

[4689 rows x 2 columns]


## Average World Length

In [15]:
def avg_word(sentence):
    words = sentence.split()
    return (sum(len(word) for word in words)/len(words))

def avg_word_length(df, col):
    df['avg_wrd'] = df[col].apply(lambda x: avg_word(x))
    print(df[[col, 'avg_wrd']].head())

In [16]:
avg_word_length(df, 'text')

                                                text    avg_wrd
0  @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...   6.142857
1  In accordance with current public health recom...   5.775000
2  The American people should know if they're sic...   5.317647
3  RT @Court_Rice LEADERSHIP: @CartwrightPA launc...   9.241379
4  https://www.wbrz.com/news/wbrz-business-direct...  76.000000


In [17]:
import nltk
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [18]:
def stop_words(df, col):
    df['stopwords'] = df[col].apply(lambda x: len([x for x in x.split() if x in stop]))
    print(df[[col, 'stopwords']].head())

In [19]:
stop_words(df, 'text')

                                                text  stopwords
0  @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...          4
1  In accordance with current public health recom...         16
2  The American people should know if they're sic...         31
3  RT @Court_Rice LEADERSHIP: @CartwrightPA launc...         11
4  https://www.wbrz.com/news/wbrz-business-direct...          0


In [20]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df2)
words = vectorizer.get_feature_names()

In [21]:
len(words)

17226

In [22]:
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(2,2))
X2 = vectorizer2.fit_transform(df2)
pairs = vectorizer2.get_feature_names()

In [23]:
len(pairs)

83221

In [24]:
from sklearn.feature_extraction.text import HashingVectorizer
vectorizer3 = HashingVectorizer(n_features=2**4)
X = vectorizer3.fit_transform(df2)
print(X.shape)

(4689, 16)


In [25]:
df3 = df2.copy
df3

<bound method NDFrame.copy of 0       @JNTHN_LCKWD @jbarro @CoryBooker Understood. W...
1       In accordance with current public health recom...
2       The American people should know if they're sic...
3       RT @Court_Rice LEADERSHIP: @CartwrightPA launc...
4       https://www.wbrz.com/news/wbrz-business-direct...
                              ...                        
4684    RT @Jenniffer2012 Esto hará disponible más rec...
4685    RT @Jenniffer2012 Envié una carta a la Adminis...
4686    RT @RepJenniffer Sent a letter to @SBAgov Admi...
4687    RT @Jenniffer2012 Excelente noticia del tesoro...
4688    RT @Jenniffer2012 Esta mañana tuve un conferen...
Name: text, Length: 4689, dtype: object>

In [ ]:
from twitter_scraper import get_tweets

for tweet in get_tweets('twitter', pages=1):
    print(tweet['text'])

In [103]:
# extract all hashtags into new column
df['hashtags'] = df.text.apply(lambda x: re.findall(r'#\S+', str(x)))
df.hashtags.head()

0    [#Shero]
1          []
2          []
3          []
4          []
Name: hashtags, dtype: object

In [104]:
df['callouts'] = df.text.apply(lambda x: re.findall(r'@\S+', str(x)))
df['callouts'].head()

0    [@RepTerriSewell]
1         [@SenateGOP]
2           [@thehill]
3                   []
4             [@WIRED]
Name: callouts, dtype: object

In [105]:
df.text.head(100)

0     RT @RepTerriSewell 62 years ago today, Rosa Pa...
1     .@SenateGOP How about we take a break until yo...
2     RT @thehill Maxine Waters tears into Trump aft...
3     It truly captures and takes action on the fron...
4     RT @WIRED Opinion: "Until the US establishes s...
5     The #GOPTaxScam doesn’t just affect taxes. It ...
6     RT @cnnbrk Ex-FBI Director James Comey tweets ...
7     RT @CRTV WATCH: @RepDeSantis shares w/ @NateMa...
8     Kate Steinle's family suffered a terrible loss...
9     "I'm completely convinced that when we pass [t...
10    RT @SolomonSyed Hear from @RepPaulTonko this S...
11    George Papadopoulos. Paul Manafort. Rick Gates...
12    RT @HouseJudDems 🚨#BREAKING: Ranking Member @R...
13    Now voting on Passage of H.R. 1699 – 5 minute ...
14    Today's development only underscores the need ...
15    In 2016, California had nearly double the numb...
16    #GOPTaxScam is riddled with special interest p...
17    #Flynn's fall is staggeringly significant 

In [106]:
charref = [x for x in df.text.apply(lambda x: re.findall(r'&[\S]+', str(x)))]
charref
flat_list = [item for sublist in charref for item in sublist]
len(flat_list)

584

In [107]:
# replace all hashtags, ampersands, and callouts with no space
df.text = df.text.apply(lambda x: re.sub(r'[#@&][\S]+', '', str(x)))
df.text.head(30)

0     RT  62 years ago today, Rosa Parks took a bold...
1     . How about we take a break until you can type...
2     RT  Maxine Waters tears into Trump after Flynn...
3     It truly captures and takes action on the fron...
4     RT  Opinion: "Until the US establishes stronge...
5     The  doesn’t just affect taxes. It will also g...
6     RT  Ex-FBI Director James Comey tweets let "ju...
7     RT  WATCH:  shares w/  his insider perspective...
8     Kate Steinle's family suffered a terrible loss...
9     "I'm completely convinced that when we pass [t...
10    RT  Hear from  this Sunday on  for more on why...
11    George Papadopoulos. Paul Manafort. Rick Gates...
12    RT  🚨 Ranking Member  After  Guilty Plea, Judi...
13    Now voting on Passage of H.R. 1699 – 5 minute ...
14    Today's development only underscores the need ...
15    In 2016, California had nearly double the numb...
16     is riddled with special interest provisions, ...
17     fall is staggeringly significant - the ti

In [108]:
# remove twitter RT and QT
df.text = df.text.apply(lambda x: re.sub(r'(\bRT.*|\bQT.*)', '', str(x)))
df.text.head(30)

0                                                      
1     . How about we take a break until you can type...
2                                                      
3     It truly captures and takes action on the fron...
4                                                      
5     The  doesn’t just affect taxes. It will also g...
6                                                      
7     \nGet  in your inbox daily for FREE:\nhttp://c...
8     Kate Steinle's family suffered a terrible loss...
9     "I'm completely convinced that when we pass [t...
10                                                     
11    George Papadopoulos. Paul Manafort. Rick Gates...
12    \n\nhttps://medium.com/the-committee-on-the-ju...
13    Now voting on Passage of H.R. 1699 – 5 minute ...
14    Today's development only underscores the need ...
15    In 2016, California had nearly double the numb...
16     is riddled with special interest provisions, ...
17     fall is staggeringly significant - the ti

In [109]:
df.text.head(30)

0                                                      
1     . How about we take a break until you can type...
2                                                      
3     It truly captures and takes action on the fron...
4                                                      
5     The  doesn’t just affect taxes. It will also g...
6                                                      
7     \nGet  in your inbox daily for FREE:\nhttp://c...
8     Kate Steinle's family suffered a terrible loss...
9     "I'm completely convinced that when we pass [t...
10                                                     
11    George Papadopoulos. Paul Manafort. Rick Gates...
12    \n\nhttps://medium.com/the-committee-on-the-ju...
13    Now voting on Passage of H.R. 1699 – 5 minute ...
14    Today's development only underscores the need ...
15    In 2016, California had nearly double the numb...
16     is riddled with special interest provisions, ...
17     fall is staggeringly significant - the ti

In [110]:
pd.set_option("display.max_rows", 200)
pd.set_option("display.max_columns", 200)

In [111]:
# remove html links
df.text = df.text.apply(lambda x:re.sub(r'(http)?', '', str(x)))
df.text.head(30)

0                                                      
1     . How about we take a break until you can type...
2                                                      
3     It truly captures and takes action on the fron...
4                                                      
5     The  doesn’t just affect taxes. It will also g...
6                                                      
7              \nGet  in your inbox daily for FREE:\n  
8     Kate Steinle's family suffered a terrible loss...
9     "I'm completely convinced that when we pass [t...
10                                                     
11    George Papadopoulos. Paul Manafort. Rick Gates...
12                                                 \n\n
13    Now voting on Passage of H.R. 1699 – 5 minute ...
14    Today's development only underscores the need ...
15    In 2016, California had nearly double the numb...
16     is riddled with special interest provisions, ...
17     fall is staggeringly significant - the ti

In [ ]:
def expand_contractions(text):
    text = contractions.fix(text)
    return text

df.text = df.text.apply(lambda x: expand_contractions(x))

In [112]:
df.text = df.text.apply(lambda x: re.sub(r'[^\w\s]', r'', str(x)))
df.text.head(30)

0                                                      
1      How about we take a break until you can type ...
2                                                      
3     It truly captures and takes action on the fron...
4                                                      
5     The  doesnt just affect taxes It will also gut...
6                                                      
7               \nGet  in your inbox daily for FREE\n  
8     Kate Steinles family suffered a terrible loss ...
9     Im completely convinced that when we pass the ...
10                                                     
11    George Papadopoulos Paul Manafort Rick Gates N...
12                                                 \n\n
13      Now voting on Passage of HR 1699  5 minute vote
14    Todays development only underscores the need t...
15    In 2016 California had nearly double the numbe...
16     is riddled with special interest provisions b...
17     fall is staggeringly significant  the tip

In [114]:
# lower casing
df.text = df.text.apply(lambda x: " ".join(x.lower() for x in x.split()))
df.text.head(30)

0                                                      
1     how about we take a break until you can type a...
2                                                      
3     it truly captures and takes action on the fron...
4                                                      
5     the doesnt just affect taxes it will also gut ...
6                                                      
7                      get in your inbox daily for free
8     kate steinles family suffered a terrible loss ...
9     im completely convinced that when we pass the ...
10                                                     
11    george papadopoulos paul manafort rick gates n...
12                                                     
13       now voting on passage of hr 1699 5 minute vote
14    todays development only underscores the need t...
15    in 2016 california had nearly double the numbe...
16    is riddled with special interest provisions bu...
17    fall is staggeringly significant the tip o

In [ ]:
text = """three cups of coffee"""
doc = nlp(text)
tokens = [w2n.word_to_num(token.text) if token.pos_ == 'NUM' else token for token in doc]

print(tokens) # result: [3, cups, of, coffee]


In [117]:
# remove numbers or words with digits  
df.text = df.text.apply(lambda x: re.sub(r'\w*\d\w*', r'', str(x)))
df.text.head(30)

0                                                      
1     how about we take a break until you can type a...
2                                                      
3     it truly captures and takes action on the fron...
4                                                      
5     the doesnt just affect taxes it will also gut ...
6                                                      
7                      get in your inbox daily for free
8     kate steinles family suffered a terrible loss ...
9     im completely convinced that when we pass the ...
10                                                     
11    george papadopoulos paul manafort rick gates n...
12                                                     
13            now voting on passage of hr   minute vote
14    todays development only underscores the need t...
15    in  california had nearly double the number of...
16    is riddled with special interest provisions bu...
17    fall is staggeringly significant the tip o

In [118]:
# remove any double or more whitespaces to single whitespaces
df.text = df.text.apply(lambda x: re.sub(r'\s\s+', ' ', str(x)))
df.text.head(30)

0                                                      
1     how about we take a break until you can type a...
2                                                      
3     it truly captures and takes action on the fron...
4                                                      
5     the doesnt just affect taxes it will also gut ...
6                                                      
7                      get in your inbox daily for free
8     kate steinles family suffered a terrible loss ...
9     im completely convinced that when we pass the ...
10                                                     
11    george papadopoulos paul manafort rick gates n...
12                                                     
13              now voting on passage of hr minute vote
14    todays development only underscores the need t...
15    in california had nearly double the number of ...
16    is riddled with special interest provisions bu...
17    fall is staggeringly significant the tip o

In [119]:
# remove any leading and trailing whitespace
df.text = df.text.apply(lambda x: re.sub(r'(\A\s+|\s+\Z)', '', str(x)))
df.text.head(30)

0                                                      
1     how about we take a break until you can type a...
2                                                      
3     it truly captures and takes action on the fron...
4                                                      
5     the doesnt just affect taxes it will also gut ...
6                                                      
7                      get in your inbox daily for free
8     kate steinles family suffered a terrible loss ...
9     im completely convinced that when we pass the ...
10                                                     
11    george papadopoulos paul manafort rick gates n...
12                                                     
13              now voting on passage of hr minute vote
14    todays development only underscores the need t...
15    in california had nearly double the number of ...
16    is riddled with special interest provisions bu...
17    fall is staggeringly significant the tip o

In [120]:
from nltk.corpus import stopwords
import string

In [121]:
stopwords_list = stopwords.words('english')
stopwords_list += list(string.punctuation)

In [ ]:
from mltk import word_tokenize

tokens = word_token